## Air Pollution

Let us use a neural network to predict the air polution at US embassy in Beijing using the following data from UCI MAchine Learning repository: https://archive.ics.uci.edu/ml/datasets/Beijing+PM2.5+Data

In [ ]:
! wget -c --retry-connrefused --tries=0 https://archive.ics.uci.edu/ml/machine-learning-databases/00381/PRSA_data_2010.1.1-2014.12.31.csv -O ~/data/workshop_data/PRSA_data_2010.1.1-2014.12.31.csv

In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from torch import nn
import torch
import time
import os

home = os.path.expanduser("~")
data = home + '/data/workshop_data/PRSA_data_2010.1.1-2014.12.31.csv'

In [ ]:
def parse_date(year, month, day, hour):
    return datetime(int(year), int(month), int(day), int(hour))

In [ ]:
df = pd.read_csv(data, parse_dates=[['year', 'month', 'day', 'hour']], date_parser=parse_date)
df = df.drop('No', axis=1)
df.columns = ['date', 'pollution', 'dew_temp', 'temp', 'pressure', 'wind_dir', 'wind_speed', 'snow', 'rain']
df.index = df['date']
df = df.drop('date', axis=1)
df = df[24:]
df['pollution'] = df['pollution'].fillna(df['pollution'].median())
df['wind_dir'] = df['wind_dir'].astype('category').cat.codes

In [ ]:
df

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
plt.figure(figsize=(16,9))
for i in range(len(df.columns)):
    plt.subplot(len(df.columns), 1, i+1)
    plt.plot(df[df.columns[i]])
    plt.title(df.columns[i], loc='right', y =.5)

## Target
We want to predict the olusion in the next our, given the previous context data.
So let us first add a target column:

In [ ]:
df['target'] = df['pollution'].shift(-1)
df = df[:-1]
features = [col for col in df.columns if col != 'target']

## Train, Test
We can not shuffle the data for random splits anymore, as we will train sequentailly.
Therefore, we define a date at which we split the data:

In [ ]:
train_end = datetime(2014, 1, 1)
#val_end = datetime(2014, 7, 1)
train = df[:train_end]
#val = df[train_end:val_end]
test = df[train_end:]

In [ ]:
def window_nd(a, window, steps = None, axis = None, outlist = False):
        """
        Create a windowed view over `n`-dimensional input that uses an 
        `m`-dimensional window, with `m <= n`.
        
    
        Parameters
        -------------
        a : Array-like
            The array to create the view on
    
        window : tuple or int
            If int, the size of the window in `axis`, or in all dimensions if 
            `axis == None`
    
            If tuple, the shape of the desired window.  `window.size` must be:
                equal to `len(axis)` if `axis != None`, else 
                equal to `len(a.shape)`, or 
                1
    
        steps : tuple, int or None
            The offset between consecutive windows in desired dimension
            If None, offset is one in all dimensions
            If int, the offset for all windows over `axis`
            If tuple, the steps along each `axis`.  
                `len(steps)` must me equal to `len(axis)`
    
        axis : tuple, int or None
            The axes over which to apply the window
            If None, apply over all dimensions
            if tuple or int, the dimensions over which to apply the window
    
        outlist : boolean
            If output should be as list of windows.  
            If False, it will be an array with 
                `a.nidim + 1 <= a_view.ndim <= a.ndim *2`.  
            If True, output is a list of arrays with `a_view[0].ndim = a.ndim`
                Warning: this is a memory-intensive copy and not a view
    
        Returns
        -------
    
        a_view : ndarray
            A windowed view on the input array `a`, or copied list of windows   
    
        """
        ashp = np.array(a.shape)
    
        if axis != None:
            axs = np.array(axis, ndmin = 1)
            assert np.all(np.in1d(axs, np.arange(ashp.size))), "Axes out of range"
        else:
            axs = np.arange(ashp.size)
    
        window = np.array(window, ndmin = 1)
        assert (window.size == axs.size) | (window.size == 1), "Window dims and axes don't match"
        wshp = ashp.copy()
        wshp[axs] = window
        assert np.all(wshp <= ashp), "Window is bigger than input array in axes"
    
        stp = np.ones_like(ashp)
        if steps:
            steps = np.array(steps, ndmin = 1)
            assert np.all(steps > 0), "Only positive steps allowed"
            assert (steps.size == axs.size) | (steps.size == 1), "Steps and axes don't match"
            stp[axs] = steps
    
        astr = np.array(a.strides)
        shape = tuple((ashp - wshp) // stp + 1) + tuple(wshp)
        strides = tuple(astr * stp) + tuple(astr)
    
        as_strided = np.lib.stride_tricks.as_strided
        a_view = np.squeeze(as_strided(a, 
                                     shape = shape, 
                                     strides = strides))
        #print(astr, strides, shape, a_view)
        if outlist:
            return list(a_view.reshape((-1,) + tuple(wshp)))
        else:
            return a_view

## Scale
Neural networks work best if the data have the same input scale. To achieve this, we will use sklearn's MinMaxScaler:

In [ ]:
df = df.astype(np.float32)
scaler = MinMaxScaler()
past_samples = 10
x_train = window_nd(scaler.fit_transform(train[features]), past_samples, 1, axis=0)
x_test = window_nd(scaler.transform(test[features]), past_samples, 1, axis=0)
y_train = scaler.fit_transform(train['target'].values.reshape(-1, 1))
y_test = scaler.transform(test['target'].values.reshape(-1, 1))

Let us start by building a [recurrent neural network](https://pytorch.org/docs/stable/nn.html#torch.nn.RNN). 
We will put the output of the recurrent layer through an activation function into a linear layer to get a single value from it. 

In [ ]:
class RNNNet(nn.Module):
    
    def __init__(self, number_of_inputs, hidden_size, num_layers):
        super().__init__()
        # Build the recurrent parts using nn.RNN
        self.rnn = 
        # Use a ReLU as an activation
        self.act = 
        # Use a linear output layer
        self.out = 
    
    def forward(self, inp):
        # implement the rest of the forward function
        output, x = self.rnn(inp)
        x = self.act(x)
        return self.out(x)

In [ ]:
def fit_batch(optim, loss, net, x, y):
    optim.zero_grad()
    y_pred = net(x)
    err = loss(y_pred, y)
    err.mean().backward()
    optim.step()
    return y_pred, err

In [ ]:
torch.cuda.manual_seed(42)
np.random.seed(42)
device = 'cpu'
if torch.cuda.is_available():
    device = 'cuda'
batch_size = 128
batches_per_epoch = 5
epochs = 20
net = RNNNet(x_train.shape[-1], 25, 1).to(device)
loss = nn.L1Loss()
optim = torch.optim.Adam(net.parameters(), lr=5e-4)
start = time.time()  
for epoch in range(epochs):
    train_err = None
    for j in range(batches_per_epoch):
        select = np.random.randint(0, len(x_train), batch_size)
        x = torch.from_numpy(x_train[select]).float().to(device)
        y = torch.from_numpy(y_train[select]).float().unsqueeze(1).to(device)
        y_pred, err = fit_batch(optim, loss, net, x, y)
        if train_err is None:
            train_err = err
        else:
            train_err += err
        #y_pred = y_pred.argmax(dim=-1)
        #acc += (y==y_pred).float().mean()
    x = torch.from_numpy(x_test).float().to(device)
    y = torch.from_numpy(y_test).float().unsqueeze(1).to(device)
    y_pred = net(x)
    test_err = loss(y_pred, y)
    print(f'Epoch {epoch} train_loss {train_err/batches_per_epoch} test_loss {test_err}')
print(f'Training time: {time.time() - start}')

Let us now use an [LSTM](https://pytorch.org/docs/stable/nn.html#torch.nn.LSTM) instead of a simple RNN.

In [ ]:
class LSTMNet(nn.Module):
    
    def __init__(self, number_of_inputs, hidden_size, num_layers):
        super().__init__()
        # Build the recurrent parts using nn.RNN
        self.rnn = 
        # Use a ReLU as an activation
        self.act = 
        # Use a linear output layer
        self.out = 
    
    def forward(self, inp):
        # implement the rest of the forward function
        output, x = self.rnn(inp)
        x = self.act(x[0])
        return self.out(x)

In [ ]:
torch.cuda.manual_seed(42)
np.random.seed(46)
net = LSTMNet(x_train.shape[-1], 25, 1).to(device)
optim = torch.optim.Adam(net.parameters(), lr=5e-4)
start = time.time()  
for epoch in range(epochs):
    train_err = None
    for j in range(batches_per_epoch):
        select = np.random.randint(0, len(x_train), batch_size)
        x = torch.from_numpy(x_train[select]).float().to(device)
        y = torch.from_numpy(y_train[select]).float().unsqueeze(1).to(device)
        y_pred, err = fit_batch(optim, loss, net, x, y)
        if train_err is None:
            train_err = err
        else:
            train_err += err
        #y_pred = y_pred.argmax(dim=-1)
        #acc += (y==y_pred).float().mean()
    x = torch.from_numpy(x_test).float().to(device)
    y = torch.from_numpy(y_test).float().unsqueeze(1).to(device)
    y_pred = net(x)
    test_err = loss(y_pred, y)
    print(f'Epoch {epoch} train_loss {train_err/batches_per_epoch} test_loss {test_err}')
print(f'Training time: {time.time() - start}')

As we are overfitting, let us add [dropout](https://pytorch.org/docs/stable/nn.html#torch.nn.Dropout) to the LSTM network:

In [ ]:
class LSTMNetWithDropout(nn.Module):
    
    def __init__(self, number_of_inputs, hidden_size, num_layers, dropout):
        super().__init__()
        # Build the recurrent parts using nn.RNN
        self.rnn = 
        # Use a ReLU as an activation
        self.act = 
        # Use a linear output layer
        self.out = 
        # Build the recurrent parts using nn.RNN
        self.dropout = 
    
    def forward(self, inp):
        # implement the rest of the forward function
        output, x = self.rnn(inp)
        x = self.act(self.dropout(x[0]))
        return self.out(x)

In [ ]:
torch.cuda.manual_seed(42)
np.random.seed(42)
net = LSTMNetWithDropout(x_train.shape[-1], 25, 1, .2).to(device)
optim = torch.optim.Adam(net.parameters(), lr=5e-4)
start = time.time()  
batches_per_epoch_new = batches_per_epoch * 4
for epoch in range(epochs):
    train_err = None
    for j in range(batches_per_epoch_new):
        select = np.random.randint(0, len(x_train), batch_size)
        x = torch.from_numpy(x_train[select]).float().to(device)
        y = torch.from_numpy(y_train[select]).float().unsqueeze(1).to(device)
        y_pred, err = fit_batch(optim, loss, net, x, y)
        if train_err is None:
            train_err = err
        else:
            train_err += err
        #y_pred = y_pred.argmax(dim=-1)
        #acc += (y==y_pred).float().mean()
    x = torch.from_numpy(x_test).float().to(device)
    y = torch.from_numpy(y_test).float().unsqueeze(1).to(device)
    y_pred = net(x)
    test_err = loss(y_pred, y)
    print(f'Epoch {epoch} train_loss {train_err/batches_per_epoch_new} test_loss {test_err}')
print(f'Training time: {time.time() - start}')